In [14]:
!pip install sentence-transformers==2.2.2

In [15]:
import json
import nltk
import string
import random
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

Download necessary NLTK data

In [16]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Loading Functions

In [17]:
def load_json(file_path):
    """
    Load a standard JSON file.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_jsonl(file_path):
    """Load a JSONL file and return a list of dictionaries."""
    try:
        with open(file_path, 'r') as file:
            return [json.loads(line) for line in file if line.strip()]  # Skip empty lines
    except Exception as e:
        print(f"Error loading JSONL file: {e}")
        return []

Preprocessing Functions

In [18]:
# Initialize Lemmatizer and Stopwords
lemmer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
remove_punct_dict = str.maketrans('', '', string.punctuation)

def preprocess(text):
    """
    Preprocess the input text by lowercasing, removing punctuation,
    tokenizing, removing stopwords, and lemmatizing.
    """
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(remove_punct_dict)
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmer.lemmatize(token) for token in tokens if token not in stop_words]
    return tokens


Load Corpus Data

In [23]:
# Define file
speaker_file = 'speakers.json'
index_file = 'index.json'
conversations_file = 'conversations.json'
utterances_file = 'utterances.jsonl'
corpus_file = 'corpus.json'  # Optional, used if you prefer the aggregated corpus

# Load JSON files
try:
    speakers = load_json(speaker_file)
    index = load_json(index_file)
    conversations = load_json(conversations_file)
except json.JSONDecodeError as e:
    print(f"Error loading JSON file: {e}")

# Load JSONL file
utterances = load_jsonl(utterances_file)

# (Optional) Load corpus.json if you prefer using it
# corpus = load_json(corpus_file)

In [24]:
print(utterances[:1])

[{'id': 'L1045', 'conversation_id': 'L1044', 'text': 'They do not!', 'speaker': 'u0', 'meta': {'movie_id': 'm0', 'parsed': [{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []}, {'tok': 'do', 'tag': 'VBP', 'dep': 'ROOT', 'dn': [0, 2, 3]}, {'tok': 'not', 'tag': 'RB', 'dep': 'neg', 'up': 1, 'dn': []}, {'tok': '!', 'tag': '.', 'dep': 'punct', 'up': 1, 'dn': []}]}]}, 'reply-to': 'L1044', 'timestamp': None, 'vectors': []}]


Create Utterance Dictionary

In [25]:
# Create a mapping from utterance_id to utterance text
utterance_dict = {utt['id']: utt['text'] for utt in utterances}

# Create a list of all utterance texts
all_utterances = [utt['text'] for utt in utterances]

Build Chatbot Components

In [26]:
# Initialize SentenceTransformer model for embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can choose other models if desired

# Precompute sentence embeddings for all utterances
print("Computing sentence embeddings...")
sentence_embeddings = model.encode(all_utterances, convert_to_tensor=True)

# Greeting definitions
GREETING_INPUTS = ("hello", "hi", "hey", "greetings", "sup", "what's up", "how are you")
GREETING_RESPONSES = ["Hi there!", "Hello!", "Hey!", "Greetings!", "How can I assist you today?"]

# Fallback responses
FALLBACK_RESPONSES = [
    "I'm not sure I understand. Could you elaborate?",
    "That's interesting! Tell me more.",
    "Can you clarify that?",
    "I'm here to help! What do you mean?"
]

def greet(sentence):
    """
    Check if the user's input is a greeting and return a greeting response.
    """
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
    return None

def fallback():
    """
    Return a random fallback response when the bot doesn't understand.
    """
    return random.choice(FALLBACK_RESPONSES)

def get_response(user_input):
    """
    Generate a response to the user's input using sentence embeddings and cosine similarity.
    """
    # Preprocess and encode the user input
    user_embedding = model.encode(user_input, convert_to_tensor=True)

    # Compute cosine similarities between user input and all utterances
    cosine_scores = cosine_similarity(
        user_embedding.cpu().numpy().reshape(1, -1),
        sentence_embeddings.cpu().numpy()
    ).flatten()

    # Find the index of the highest similarity score
    top_idx = np.argmax(cosine_scores)
    top_score = cosine_scores[top_idx]

    # Define a similarity threshold
    threshold = 0.5  # You can adjust this value based on testing

    if top_score < threshold:
        return fallback()
    else:
        return all_utterances[top_idx]


Computing sentence embeddings...


Chatbot Interaction Loop

In [27]:
def chat():
    """
    Start the chatbot interaction loop.
    """
    print("Bot: Hello! I am the retrieval learning bot. Start typing your message after a greeting to talk to me. To end the conversation, type 'bye'!")

    while True:
        user_input = input("You: ").strip().lower()

        if user_input == 'bye':
            print("Bot: Goodbye! Take care.")
            break
        elif user_input in ['thanks', 'thank you']:
            print("Bot: You're welcome!")
            continue

        # Check for greeting
        greeting_response = greet(user_input)
        if greeting_response:
            print(f"Bot: {greeting_response}")
            continue

        # Generate a response
        response = get_response(user_input)
        print(f"Bot: {response}")

Main Execution

In [28]:
if __name__ == "__main__":
    chat()


Bot: Hello! I am the retrieval learning bot. Start typing your message after a greeting to talk to me. To end the conversation, type 'bye'!
You: hi
Bot: Hi there!
You: how are you
Bot: How are you?
You: i will kill you
Bot: I'll kill you!
You: shut up
Bot: Shut up.
You: why you are copying me
Bot: Why are you following me?
You: help me
Bot: Help me.
You: what is this behaviour?
Bot: Why is this?
You: you are dumb
Bot: You're being stupid.
You: bye
Bot: Goodbye! Take care.
